In [709]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as sc
import io
import requests
import statsmodels.api as sm

url1 = "https://raw.githubusercontent.com/SzMatej/IAU_2020-2021/main/65/other_train.csv"
url2 = "https://raw.githubusercontent.com/SzMatej/IAU_2020-2021/main/65/personal_train.csv"

db1 = requests.get(url1).content
db2 = requests.get(url2).content
train = pd.read_csv(io.StringIO(db1.decode('utf-8')))
other = pd.read_csv(io.StringIO(db2.decode('utf-8')))

In [710]:
data = pd.merge(train,other,on=['name','address'], how = 'outer')

print(len(data))

3983


In [711]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3983 entries, 0 to 3982
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0_x    3983 non-null   int64  
 1   name            3983 non-null   object 
 2   address         3983 non-null   object 
 3   race            3960 non-null   object 
 4   marital-status  3967 non-null   object 
 5   occupation      3962 non-null   object 
 6   pregnant        3969 non-null   object 
 7   education-num   3569 non-null   float64
 8   relationship    3967 non-null   object 
 9   capital-gain    3963 non-null   float64
 10  education       3968 non-null   object 
 11  fnlwgt          3965 non-null   float64
 12  class           3964 non-null   float64
 13  income          3968 non-null   object 
 14  medical_info    3965 non-null   object 
 15  native-country  3969 non-null   object 
 16  hours-per-week  3961 non-null   float64
 17  capital-loss    3968 non-null   f

### atribút pregnant

Upravíme boolean hodnoty v datasete tak že ich nahradíme číselnou reprezentáciou.
Taktiež nastavíme všetky hodnoty tohto atribútu u mužov na 0.

In [712]:
def sanitize_boolean(boolean):
    try:
        if boolean.strip() in ['f','F','FALSE','false','False']:
            return 0
        elif boolean.strip() in ['t','T','TRUE','true','True']:
            return 1
        else:
            return np.nan
    except AttributeError:
        return np.nan

In [713]:
data.pregnant = data.pregnant.map(sanitize_boolean)

In [714]:
def sanitize_pregnancy(data):
    data.loc[(data.sex == 1),'pregnant'] = 0
    return data

data = sanitize_pregnancy(data)

### atribút sex

Tento atribút neobsahuje žiadne prázdne hodnoty. Pri tomto atribúte treba len dané hodnoty prekonvertovať na numerické hodnoty.

In [715]:
data.sex.unique()

array([' Male', ' Female'], dtype=object)

In [716]:
def sanitize_sex(sex):
    return 1 if sex.strip() == 'Male' else 0

In [717]:
data.sex = data.sex.map(lambda sex: sanitize_sex(sex))

In [718]:
data.sex.unique()

array([1, 0], dtype=int64)

Hodnoty Male sme nahradili hodnotou 1 a naopak hodnotu female sme nahradili hodnotou 0

### atribút age

Pri tomto atribúte nahradíme záporné hodnoty na NaN. Takisto sa tu nachádzajú aj neznáme hodnoty ktoré sú reprezentované '??'. Tieto hodnoty taktiež nahradíme hodnotami NaN. Ďalej aby mali hodnoty jednotný dátový typ tak ich prekonvertujeme na numerickú hodnotu.

In [719]:
data.age.unique()

array(['53', '55', '65', '66', '37', '??', '42', nan, '57', '43', '56',
       '25', '45', '58', '86', '48', '54', '67', '46', '73', '71', '59',
       '64', '38', '62', '26', '70', '50', '52', '63', '61', '31', '30',
       '29', '74', '83', '41', '69', '75', '49', '27', '44', '51', '34',
       '60', '36', '40', '39', '15', '47', '20', '35', '32', '21', '68',
       '78', '77', '33', '28', '19', '80', '18', '81', '85', '17', '76',
       '9', '72', '7', '87', '24', '90', '3', '23', '79', '16', '22',
       '82', '84', '14', '113', '93', '-1', '12', '99'], dtype=object)

In [720]:
def sanitize_age(data):
    data.loc[(data.age == '-1'),'age'] = np.nan
    data.loc[(data.age == '??'),'age'] = np.nan
    return data

In [721]:
data = sanitize_age(data)

In [722]:
def sanitize_number(number):
    try:
        sanitized = int(pd.to_numeric(number, errors="coerce"))
        return sanitized if sanitized > 0 else np.nan
    except AttributeError:
        return np.nan
    except ValueError:
        return np.nan

In [723]:
data.age = data.age.map(lambda age: sanitize_number(age))

In [724]:
data.age.unique()

array([ 53.,  55.,  65.,  66.,  37.,  nan,  42.,  57.,  43.,  56.,  25.,
        45.,  58.,  86.,  48.,  54.,  67.,  46.,  73.,  71.,  59.,  64.,
        38.,  62.,  26.,  70.,  50.,  52.,  63.,  61.,  31.,  30.,  29.,
        74.,  83.,  41.,  69.,  75.,  49.,  27.,  44.,  51.,  34.,  60.,
        36.,  40.,  39.,  15.,  47.,  20.,  35.,  32.,  21.,  68.,  78.,
        77.,  33.,  28.,  19.,  80.,  18.,  81.,  85.,  17.,  76.,   9.,
        72.,   7.,  87.,  24.,  90.,   3.,  23.,  79.,  16.,  22.,  82.,
        84.,  14., 113.,  93.,  12.,  99.])

In [725]:
len(data[pd.isnull(data.age)])

400

Sem vidíme že pri tomto atribúte sa vyskytuje furt veľa chýbajúcich hodnôt (400). Toto sa dá vyriešiť niekoľkými spôsobmi. Buď vieme túto hodnotu nahradiť priemerom podľa jeho vekovej kategórie čo je lahší spôsob alebo vieme túto hodnotu dopočítať podľa jeho dátumu narodenia.

### atribút medical info

Pri tomto atribúte treba vyparsovať hodnoty jedného riadku a následne vytvoriť nové stĺpce do ktorých sa táto vyparsovaná hodnota doplní

In [726]:
data.medical_info.unique()

array(["{'mean_glucose':'111.8125','std_glucose':'44.88174566','kurtosis_glucose':'0.423867091','skewness_glucose':'0.239944007','mean_oxygen':'2.465719064','std_oxygen':'17.28981747','kurtosis_oxygen':'8.636118173','skewness_oxygen':'85.60462065'}",
       "{'mean_glucose':'71.3984375','std_glucose':'47.29517349','kurtosis_glucose':'1.317459045','skewness_glucose':'2.340411994','mean_oxygen':'17.114548499999998','std_oxygen':'46.86283003','kurtosis_oxygen':'3.070346153','skewness_oxygen':'9.405187883'}",
       "{'mean_glucose':'102.796875','std_glucose':'37.5346422','kurtosis_glucose':'0.382097319','skewness_glucose':'1.3556516','mean_oxygen':'2.706521739','std_oxygen':'19.87410184','kurtosis_oxygen':'7.955209532','skewness_oxygen':'67.64902246'}",
       ...,
       "{'mean_glucose':'129.90625','std_glucose':'48.62217818','kurtosis_glucose':'0.038773097','skewness_glucose':'-0.17251288','mean_oxygen':'3.319397993','std_oxygen':'22.72505413','kurtosis_oxygen':'8.750496392','skewness_

In [727]:
x = data['medical_info'].str.replace('{','').str.replace('\'','').str.split(',', expand=True)
i = 0
while i < len(x.columns):
    y = x[i].str.split(':', expand=True)
    data[y[0][0]] = y[1].astype(float)
    i += 2
data.drop('medical_info', axis='columns', inplace=True)

### atribút date of birth

Tento atribút nemá nejaký jednotný tvar, čiže je nutné to ošetriť

In [728]:
data.date_of_birth.unique()

array(['1966-05-16', '1964-06-29', '54-01-25', ..., '1988-09-29',
       '1958/12/17', '1959-09-29'], dtype=object)

In [729]:
def sanitize_date(date):
    date = str(date).replace('/', '-')
    date = date[:10]
    date = date.split("-")
    
    if date[0] != 'nan':
        if len(date[0]) != 4:
            if len(date[2]) == 2 and int(date[0]) > 31:
                new_date = "19"+ date[0] +"-"+date[1]+"-"+date[2] 
                
            elif ((len(date[2]) == 2) and (int(date[0]) < 31) and (int(date[2]) > 31)):
                new_date = "19"+date[2]+"-"+date[1]+"-"+date[0] 
                
            elif ((len(date[2]) == 2) and (int(date[0]) < 31) and (int(date[2]) < 31)):
                new_date = "20"+ date[2] + "-" +date[1]+"-" + date[0] 
            else:
                new_date = date[2]+"-"+date[1]+"-"+date[0] 
            return new_date        
    return '-'.join(date)

In [730]:
data.date_of_birth = data.date_of_birth.map(sanitize_date)
data.date_of_birth.head(10)

0    1966-05-16
1    1964-06-29
2    1954-01-25
3    1952-10-30
4    1982-01-13
5    1950-09-17
6    1976-11-25
7    1954-07-14
8    1954-07-14
9    1962-09-11
Name: date_of_birth, dtype: object

Teraz má tento atribúť jednotný tvar hodnôt. teraz môžme upraviť atribút age tak, že pomocou dátumu vypočítame chýbajúce hodnoty

In [731]:
len(data[pd.isnull(data.date_of_birth)])

0

Tu vidíme že pri stĺpci dátumu narodenia nechýbajú žiadne hodnoty tak by nemal byť problém s dopočítaním hodnôt pre stĺpec vek

In [732]:
years = [];

def find_measure_year(age, date_of_birth):
    if (pd.notnull(age)):
        years.append(int(date_of_birth.split('-')[0]) + int(age))
        
data.apply(lambda x: find_measure_year(x.age,x.date_of_birth), axis = 1)

measure_year = pd.Series(years).mode();

In [733]:
def fill_null_age(age,date):
    if (date):
        return (measure_year - int(date.split('-')[0]))[0]
    return np.nan

data.loc[data.age.isnull(),'age'] = data[data.age.isnull()].apply(lambda x: fill_null_age(x.age,x.date_of_birth),axis = 1)

In [734]:
len(data[pd.isnull(data.age)])

0

Ako je vidieť podarilo sa nám doplniť všetky hodnoty pri stĺpci age. Docielili sme to tak, že sme našli najviac vyskytujúci sa rok po sčítani veku a roku narodenia. Takto sme našli rok kedy sa uskutočnilo meranie, podľa ktorého sme ďalej našli roky chýbajúcich ľudí v datasete podľa dátumu narodenia.

In [735]:
data.age.unique()

array([ 53.,  55.,  65.,  66.,  37.,  69.,  42.,  57.,  43.,  56.,  25.,
        45.,  58.,  86.,  48.,  54.,  67.,  46.,  73.,  71.,  59.,  64.,
        38.,  62.,  26.,  70.,  50.,  52.,  63.,  61.,  51.,  31.,  30.,
        72.,  29.,  74.,  83.,  41.,  75.,  49.,  27.,  44.,  34.,  60.,
        36.,  40.,  39.,  32.,  15.,  47.,  68.,  20.,  35.,  21.,  78.,
        77.,  33.,  28.,  19.,  80.,  18.,  81.,  85.,  17.,  76.,   9.,
         7.,  87.,  24.,  90.,   3.,  23.,  79.,  16.,  97.,  22.,  82.,
        84.,  14., 113.,  93.,  -1.,  12.,  99.])

Po úprave vidíme že sa pri atrigúte age znova vyskytuje hodnota -1. To znamená len to, že sa v datasete vyskytujú aj niektoré zlé dátumy. Tieto hodnoty teda doplníme priemerom ako bolo spomenuté vyššie.

In [736]:
data.loc[(data.age < 0)].date_of_birth

3307    2020-04-06
Name: date_of_birth, dtype: object

Tu vidíme, že pri niektorých hodnotách je naozaj nastavený zlý dátum narodenia a to ten istý

In [737]:
median = data[(data.age > 0)].groupby('sex', as_index=False).age.mean()
data.loc[(data.age < 0), 'age'] = data[data.age < 0].age.map(lambda a: round(median.loc[1, 'age'], 0))

In [738]:
data['age'].describe()

count    3983.000000
mean       51.844590
std        11.783656
min         3.000000
25%        44.000000
50%        52.000000
75%        60.000000
max       113.000000
Name: age, dtype: float64

In [739]:
data.age.unique()

array([ 53.,  55.,  65.,  66.,  37.,  69.,  42.,  57.,  43.,  56.,  25.,
        45.,  58.,  86.,  48.,  54.,  67.,  46.,  73.,  71.,  59.,  64.,
        38.,  62.,  26.,  70.,  50.,  52.,  63.,  61.,  51.,  31.,  30.,
        72.,  29.,  74.,  83.,  41.,  75.,  49.,  27.,  44.,  34.,  60.,
        36.,  40.,  39.,  32.,  15.,  47.,  68.,  20.,  35.,  21.,  78.,
        77.,  33.,  28.,  19.,  80.,  18.,  81.,  85.,  17.,  76.,   9.,
         7.,  87.,  24.,  90.,   3.,  23.,  79.,  16.,  97.,  22.,  82.,
        84.,  14., 113.,  93.,  12.,  99.])

Tu boli nastavené všetky tieto neznáme hodnoty mediánom podľa pohlavia.

### atrbút income

Pri tomto atribúte tiež prekonvertujeme hodnoty na číselný tvar

In [740]:
data.income.unique()

array([' >50K', ' <=50K', nan], dtype=object)

In [741]:
data.income = data.income.map(lambda income: 0 if str(income).strip() == '<=50K' else 1 if str(income).strip() == '>50K' else np.nan)

In [742]:
data.income.unique()

array([ 1.,  0., nan])

### odstránenie duplikátov

In [743]:
print('other lenght:',len(other))
print('personal lenght: ',len(train))
print('data lenght(spojený dataset): ',len(data))

other lenght: 3933
personal lenght:  3983
data lenght(spojený dataset):  3983


Tu môžme vidieť že tieto dva datasety nemajú rovnakú dĺžku a pritom by ju mali mať rovnakú. Tak v prvom rade odstránime možné duplikáty v spojenom datasete, ktorý sme spojili na začiatku. Odstraňovať ich budeme na základe mena, dátumu narodenia a adresy.

In [744]:
data = data.drop_duplicates(['name','address','date_of_birth'], keep="last")
print('data lenght(spojený dataset): ',len(data))

data lenght(spojený dataset):  3933


### pridanie chýbajúcich hodnôt

In [745]:
data[data.columns[data.isnull().any()].tolist()].isnull().sum()

race                 11
marital-status        7
occupation           13
pregnant              7
education-num       400
relationship          9
capital-gain         10
education            10
fnlwgt               11
class                12
income                9
native-country        8
hours-per-week        9
capital-loss          7
workclass             8
mean_glucose          9
kurtosis_glucose      9
mean_oxygen           9
kurtosis_oxygen       9
dtype: int64

Sem vidíme, v ktorých stĺpcoch datasetu chýba koľko hodnôt. Na doplnenie týchto hodnot sa dá využit viacero ktoré využujeme (medián, knn, lineárna regresia).

### doplnenie hodnôt education-num

V prvej analýze sme zistili závislosť atribútov education a atribútov education-num, kde education-num je len čiselná reprezentácia stĺpca education. Čiže pri tomto atribúte nám stačí doplniť túto hodnotu podľa stĺpca education.

In [746]:
def get_education_num(education):
    temp = 10.0
    for i in data['education-num'].loc[data.education == education]:
        if (i > 0) & (i < temp):
            temp = i
    return temp


map = {}
for i in data.education.unique():
    map[i] = (get_education_num(i))
print(map)

{' Masters': 10.0, ' Some-college': 10.0, ' Bachelors': 10.0, ' HS-grad': 9.0, ' 7th-8th': 4.0, ' Assoc-voc': 10.0, ' Prof-school': 10.0, nan: 10.0, ' 11th': 7.0, ' 9th': 5.0, ' 10th': 6.0, ' Assoc-acdm': 10.0, ' 12th': 8.0, ' 5th-6th': 3.0, ' Doctorate': 10.0, ' 1st-4th': 2.0, ' Preschool': 1.0}


In [747]:
data.loc[:,'education-num'] = data.education.map(map)

In [748]:
print('Missing education-num: ', len(data[data['education-num'].isnull()]))
print('Missing education: ', len(data[data['education'].isnull()]))

Missing education-num:  0
Missing education:  10


Tieto hodnoty sa nám podarilo doplniť až nazopár hodnôt pri ktorých nie je uvedená ani hodnota education.

### Attribút Unnamed: 0_x 
obsahuje poradové čísla, ktoré nepotrebujeme

In [749]:
print(data['Unnamed: 0_x'].unique())
data.drop('Unnamed: 0_x', axis='columns', inplace=True)

[   0    1    2 ... 3980 3981 3982]


### Attribut Unnamed: 0_y
nevieme co reprezentuje, zmazeme ho

In [750]:
print(data['Unnamed: 0_y'].unique())
data.drop('Unnamed: 0_y', axis='columns', inplace=True)

[3768 1089 3071 ... 3745 3735 2650]


### Attribut fnlwgt
nevieme co reprezentuje, zmazeme ho

In [751]:
print(data['fnlwgt'].unique())
data.drop('fnlwgt', axis='columns', inplace=True)

[270092.  34278. 107302. ... 217961.  75167. 202056.]


### Stringove attributy 

In [752]:
def sanitize_string(string):
    try:
        string = string.strip()
        if string in ['None', 'nan', '??', '?']:
            return np.nan
        return string
    except AttributeError:
        return np.nan

### race

In [753]:
data['race'] = data['race'].map(sanitize_string)
data.race.value_counts()

White                 3328
Black                  394
Asian-Pac-Islander     131
Amer-Indian-Eskimo      41
Other                   28
Name: race, dtype: int64

Hodnota Asian-Pac-Islander a Amer-Indian-Eskimo reprezentuju rozne rasy, tak ich dame do Other

In [754]:
data.race = data.race.map(lambda race: 'Other' if race in ('Asian-Pac-Islander', 'Amer-Indian-Eskimo') else race)

In [755]:
data.race.value_counts()

White    3328
Black     394
Other     200
Name: race, dtype: int64

### marital-status, relationship
Tieto attributy reprezentuju podobne hodnoty, chceme ich pocistit a zjednotit

In [756]:
data['marital-status'] = data['marital-status'].map(sanitize_string)
data['relationship'] = data['relationship'].map(sanitize_string)

In [757]:
data['marital-status'].unique()

array(['Married-civ-spouse', 'Never-married', 'Separated', 'Divorced',
       'Married-spouse-absent', 'Widowed', 'Married-AF-spouse', nan],
      dtype=object)

In [758]:
data['relationship'].unique()

array(['Husband', 'Not-in-family', 'Unmarried', 'Wife', 'Own-child', nan,
       'Other-relative'], dtype=object)

Po analyze sme urcili, ze vo vyslednom attribute relationship budu nasledujuce hodnoty: Not-Married, Married, Divorced/Widowed
Pri hodnotach, ako unmarried nevieme urcit, ci su vo vztahu, preto pouzivame vyraz Not-Married.

In [759]:
def sanitize_relationship(relation):
    if relation in ('Husband', 'Wife'):
        return 'Married'
    if relation in ('Not-in-family', 'Unmarried', 'Own-child', 'Other-relative'):
        return 'Not-Married'
    return relation
def sanitize_marital_status(married):
    if married in ('Married', 'Married-civ-spouse', 'Married-spouse-absent', 'Married-AF-spouse'):
        return 'Married'
    if married in ('Not-Married', 'Never-married', 'Separated', 'Divorced', 'Widowed'):
        return 'Not-Married'
    if married in ('Separated', 'Divorced', 'Widowed'):
        return 'Divorced/Widowed'
    return np.nan

In [778]:
data['relationship'] = data['relationship'].map(sanitize_relationship)
data['relationship'] = data['marital-status'].map(sanitize_marital_status)
data.drop('marital-status', axis='columns', inplace=True)

In [777]:
data['relationship'].unique()

array(['Married', 'Not-Married', nan], dtype=object)

In [763]:
data['occupation'].unique()

array([' Prof-specialty', ' Sales', ' Adm-clerical', ' Craft_repair',
       ' Prof_specialty', ' Priv-house-serv', ' ?', ' Craft-repair',
       ' Other-service', ' Exec-managerial', ' Farming-fishing',
       ' Machine_op_inspct', ' Transport_moving', ' Handlers-cleaners',
       ' Transport-moving', ' Tech-support', ' Other_service',
       ' Exec_managerial', ' Farming_fishing', ' Machine-op-inspct',
       ' Protective-serv', ' Tech_support', nan, ' Armed-Forces',
       ' Adm_clerical', ' Priv_house_serv', ' Handlers_cleaners',
       ' Protective_serv'], dtype=object)

In [764]:
data['education'].unique()

array([' Masters', ' Some-college', ' Bachelors', ' HS-grad', ' 7th-8th',
       ' Assoc-voc', ' Prof-school', nan, ' 11th', ' 9th', ' 10th',
       ' Assoc-acdm', ' 12th', ' 5th-6th', ' Doctorate', ' 1st-4th',
       ' Preschool'], dtype=object)

In [765]:
data['native-country'].unique()

array([' United-States', ' ?', nan, ' Poland', ' Hungary',
       ' Dominican-Republic', ' El-Salvador', ' Mexico', ' Philippines',
       ' Italy', ' China', ' Laos', ' Scotland', ' Canada', ' Germany',
       ' Taiwan', ' France', ' Vietnam', ' England', ' Columbia',
       ' Cambodia', ' Iran', ' Puerto-Rico', ' Japan', ' Haiti',
       ' Thailand', ' Nicaragua', ' Hong', ' Cuba', ' Greece', ' Ecuador',
       ' Jamaica', ' South', ' Guatemala', ' India', ' Portugal', ' Peru',
       ' Trinadad&Tobago', ' Ireland', ' Honduras', ' Yugoslavia'],
      dtype=object)

In [766]:
data['workclass'].unique()

array([' Private', ' Federal-gov', ' Self-emp-not-inc', nan, ' ?',
       ' Self-emp-inc', 'local-gov', ' Local-gov', ' State-gov',
       'private', 'self-emp-not-inc', '?', 'self-emp-inc', 'federal-gov',
       'state-gov', ' Never-worked'], dtype=object)

In [767]:
data['occupation'] = data['occupation'].map(sanitize_string)
data['education'] = data['education'].map(sanitize_string)
data['native-country'] = data['native-country'].map(sanitize_string)
data['workclass'] = data['workclass'].map(sanitize_string)

In [768]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3933 entries, 0 to 3982
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              3933 non-null   object 
 1   address           3933 non-null   object 
 2   race              3922 non-null   object 
 3   marital-status    3926 non-null   object 
 4   occupation        3679 non-null   object 
 5   pregnant          3926 non-null   float64
 6   education-num     3933 non-null   float64
 7   relationship      3926 non-null   object 
 8   capital-gain      3923 non-null   float64
 9   education         3923 non-null   object 
 10  class             3921 non-null   float64
 11  income            3924 non-null   float64
 12  native-country    3857 non-null   object 
 13  hours-per-week    3924 non-null   float64
 14  capital-loss      3926 non-null   float64
 15  workclass         3684 non-null   object 
 16  age               3933 non-null   float64


In [769]:
data['marital-status'].unique()

array(['Married-civ-spouse', 'Never-married', 'Separated', 'Divorced',
       'Married-spouse-absent', 'Widowed', 'Married-AF-spouse', nan],
      dtype=object)

In [770]:
data['relationship'].unique()

array(['Married', 'Not-Married', nan], dtype=object)